# 几种波动率的计算

July 17, 2024

```{contents}
```

## 1. 历史波动率

### 1.1 基于收盘价的波动率计算

历史波动率是基于过去价格数据计算得出的波动率。它反映了资产在过去一定时期内的价格变动幅度。通常通过计算资产价格的标准差来估计历史波动率。

- **计算方法**：使用过去价格的对数收益率计算标准差。
- **公式**：

  $$
  \sigma_{hist}^2 = \frac{1}{N-1} \sum_{i=1}^{N} (r_i - \bar{r})^2
  $$
  
    其中 $ r_i $ 是第 $ i $ 个收益率，$ \bar{r} $ 是收益率的平均值，$ N $ 是样本数量。
    
    

- 年化波动率的计算公式为：

  $$
  \sigma_{\text{annual}} = \sigma_{\text{daily}} \times \sqrt{252}
  $$
  
  其中，252是交易日的平均数量。通常，日收益率的标准差需要转换为年化波动率。


### 1.2 Garman-Klass波动率模型

Garman-Klass波动率模型在1980年由Garman和Klass提出，提供了一个更全面的波动率估计，适用于高频数据和对波动性变化敏感的分析。它用于改进基于高、低、开盘和收盘价的波动率估计。公式如下：

$$
\sigma^2={0.511*(H_t-L_t)^2-0.019*[(C_t-O_t)(H_t+L_t-2*O_t)-2*(H_t-O_t)*(L_t-O_t)] \\
-0.383*(C_t-O_t)^2}
$$

其中H、L、O和C分别表示最高价、最低价、开盘价和收盘价。从计算的波动率时间序列来看，与VIX指数走势基本一致。

它有以下优点：
- 适用于基于日内高频数据（如每分钟、每小时）的波动率估计
- 使用了修正系数（0.511、-0.019、-0.383）来调整不同价格对波动率的贡献

In [3]:
import numpy as np
import pandas as pd
def garman_klass_volatility(open_, high, low, close):
    term1 = 0.511 * (high - low) ** 2
    term2 = -0.019 * ((close - open_) * (high + low - 2 * open_) - 2 * (high - open_) * (low - open_))
    term3 = -0.383 * (close - open_) ** 2
    volatility = np.sqrt(term1 + term2 + term3)
    return volatility

## 2. 条件波动率
条件波动率 (Conditional Volatility)是基于金融时间序列模型（如GARCH模型）计算的波动率。与历史波动率（基于过去价格数据计算）和隐含波动率（从期权价格推导）不同，条件波动率依赖于动态模型来更新对未来波动性的预测，考虑了历史数据的影响。

### 2.1 GARCH模型
GARCH (Generalized Autoregressive Conditional Heteroskedasticity) 模型由Tim Bollerslev于1986年提出，扩展了Arch（自回归条件异方差）模型

**GARCH(1,1)模型**的基本形式为：



$$
\sigma_t^2 = \alpha_0 + \alpha_1 \epsilon_{t-1}^2 + \beta_1 \sigma_{t-1}^2
$$
    


其中：
- $\sigma_t^2$ 是条件方差（波动率的平方）。
- $\epsilon_{t-1}^2$ 是前期的误差平方。
- $\sigma_{t-1}^2$ 是前期的条件方差。
- $\alpha_0$、$\alpha_1$ 和 $\beta_1$ 是模型参数。



### 2.2 EGARCH 模型
EGARCH(Exponential GARCH) 模型扩展了GARCH模型，由 Nelson (1991) 提出。EGARCH 模型允许波动率的对数形式以对数线性方式来建模，从而可以捕捉到金融时间序列中常见的波动率不对称性（例如，负面消息对波动率的影响大于正面消息）。
  - 公式为：
    $$
    \ln(\sigma_t^2) = \alpha_0 + \alpha_1 \left(\frac{\epsilon_{t-1}}{\sigma_{t-1}}\right) + \beta_1 \ln(\sigma_{t-1}^2)
    $$
    其中，$\ln(\sigma_t^2)$是条件波动率的对数。


以下是如何使用Python中的`arch`库计算条件波动率的示例代码：

In [ ]:
import pandas as pd
import numpy as np

def calculate_log_returns(df):
    """
    计算对数收益率
    df: 收盘价
    """
    return np.log(df / df.shift(1)).dropna()

In [ ]:
log_returns = calculate_log_returns(close_df)

In [ ]:
def estimate_volatility(series):
    """
    计算基于Garch模型的条件波动率
    series: 单条对数收益率序列
    """
    model = arch_model(series, vol='Garch', p=1, q=1)
    res = model.fit(disp='off')
    return res.conditional_volatility

## Reference

1. Bollerslev, T., 1986. Generalized autoregressive conditional heteroskedasticity. Journal of Econometrics 31, 307–327. https://doi.org/10.1016/0304-4076(86)90063-1
1. Diebold, F.X., Yilmaz, K., 2009. Measuring Financial Asset Return and Volatility Spillovers, with Application to Global Equity Markets. The Economic Journal 119, 158–171. https://doi.org/10.1111/j.1468-0297.2008.02208.x
1. Engle, R.F., 1982. Autoregressive Conditional Heteroscedasticity with Estimates of the Variance of United Kingdom Inflation. Econometrica 50, 987. https://doi.org/10.2307/1912773
1. Garman, M.B., Klass, M.J., 1980. On the Estimation of Security Price Volatilities from Historical Data. J BUS 53, 67. https://doi.org/10.1086/296072
1. Nelson, D.B., 1991. Conditional Heteroskedasticity in Asset Returns: A New Approach. Econometrica 59, 347. https://doi.org/10.2307/2938260
